# S3 creation and uploaded files for fine tunning

In [1]:
import os
import json
import boto3

In [2]:
# Initialize the S3 client
s3_client = boto3.client('s3')

# Define the bucket name and local images folder
bucket_name = 'finetunningforimagegeneration'
IMAGES_FOLDER = 'images'
METADATA_FILE_NAME = 'metadata.jsonl'
METADATA_FILE_PATH = os.path.join(IMAGES_FOLDER, METADATA_FILE_NAME)

In [18]:
import boto3
from botocore.exceptions import ClientError

def create_bucket(bucket_name, region='us-east-1'):
    # Create an S3 client
    s3_client = boto3.client('s3', region_name=region)
    
    # Check if the bucket already exists
    try:
        s3_client.head_bucket(Bucket=bucket_name)
        print(f'Bucket {bucket_name} already exists.')
        return True
    except ClientError as e:
        error_code = e.response['Error']['Code']
        if error_code == '404':
            # Bucket does not exist, proceed to create it
            pass
        else:
            print(f'Error checking bucket: {e}')
            return False
    
    # Create the bucket
    try:
        if region == 'us-east-1':
            s3_client.create_bucket(Bucket=bucket_name)
        else:
            s3_client.create_bucket(Bucket=bucket_name, CreateBucketConfiguration={'LocationConstraint': region})
        print(f'Bucket {bucket_name} created successfully.')
    except ClientError as e:
        print(f'Error creating bucket: {e}')
        return False
    return True

# Example usage
bucket_name = 'finetunningforimagegeneration'  # Replace with your desired bucket name
region = 'us-east-1'  # Replace with your desired region, e.g., 'us-west-1'

create_bucket(bucket_name, region)


Bucket finetunningforimagegeneration already exists.


True

In [3]:
# Ensure the images folder exists
if not os.path.exists(IMAGES_FOLDER):
    print(f"The folder {IMAGES_FOLDER} does not exist.")
    exit()

In [11]:
# List all image files in the images folder
image_files = [f for f in os.listdir(IMAGES_FOLDER) if f.endswith(('png', 'jpg', 'jpeg'))]

# Load the metadata.jsonl file
metadata = []
with open(METADATA_FILE_PATH, 'r') as f:
    for line in f:
        stripped_line = line.strip()
        if stripped_line:  # Ensure the line is not empty
            try:
                metadata.append(json.loads(stripped_line))
            except json.JSONDecodeError as e:
                print(f"Error decoding JSON on line: {line}")
                print(f"Error message: {e}")
                continue  # Skip lines that cannot be decoded

In [12]:
# Function to upload a file to S3
def upload_file_to_s3(file_path, bucket_name, object_name):
    try:
        s3_client.upload_file(file_path, bucket_name, object_name)
        return f"s3://{bucket_name}/{object_name}"
    except Exception as e:
        print(f"Error uploading {file_path}: {e}")
        return None

In [19]:
# Upload images and update metadata with S3 paths
updated_metadata = []

for item in metadata:
    image_file = item["image-ref"]
    file_path = os.path.join(IMAGES_FOLDER, image_file)
    
    if os.path.exists(file_path):
        s3_path = upload_file_to_s3(file_path, bucket_name, f"fine_tuning_images/{image_file}")
        if s3_path:
            item["image-ref"] = s3_path
            updated_metadata.append(item)
    else:
        print(f"File {file_path} does not exist.")

In [20]:
# Save the updated metadata.jsonl file locally
updated_metadata_file_path = os.path.join(IMAGES_FOLDER, 'updated_metadata.jsonl')
with open(updated_metadata_file_path, 'w') as jsonl_file:
    for entry in updated_metadata:
        jsonl_file.write(json.dumps(entry) + '\n')

# Upload the updated metadata.jsonl file to S3
upload_file_to_s3(updated_metadata_file_path, bucket_name, f"fine_tuning_images/{os.path.basename(updated_metadata_file_path)}")

print("All images and the updated JSONL file have been uploaded to S3 successfully.")


All images and the updated JSONL file have been uploaded to S3 successfully.


# fine tunning Amazon titan imagae generation G1 model

In [51]:
import boto3

# Initialize the Bedrock client
bedrock = boto3.client(service_name='bedrock', region_name='us-east-1')

# Job details
job_name = 'fine-tuning-titan-image-generation-job'
model_name = 'fine-tuned-titan-image-model'
role_arn = 'arn:aws:iam::236515146491:role/TitanImageGenerationFineTunning'  # Replace with your actual role ARN

# Base model ID for the Titan Image Generation model
base_model_id = "**amazon.titan-image-generator-v1**" 

# Training Data S3 URI
training_data_s3_uri = "s3://finetunningforimagegeneration/fine_tuning_images/updated_metadata.jsonl"

# Output Data S3 URI
output_data_s3_uri = "s3://finetunningforimagegeneration/output/"

# Number of steps based on 23 images
num_steps = 4000  # Setting the number of steps based on AWS recommendations for 23 images


In [39]:
# Hyperparameters based on AWS recommendations
hyperparameters = {
    "steps": str(num_steps),  # Number of steps
    "batchSize": "8",         # Batch size
    "learningRate": "0.00001" # Learning rate
}

In [43]:
# Create model customization job
response = bedrock.create_model_customization_job(
    customizationType="FINE_TUNING",
    jobName=job_name,
    customModelName=model_name,
    roleArn=role_arn,
    baseModelIdentifier=base_model_id,
    hyperParameters=hyperparameters,
    trainingDataConfig={"s3Uri": training_data_s3_uri},
    outputDataConfig={"s3Uri": output_data_s3_uri}  # Output S3 path
)

print("Model customization job created:", response)

ValidationException: An error occurred (ValidationException) when calling the CreateModelCustomizationJob operation: The provided model identifier is invalid.

In [7]:
import boto3
from botocore.exceptions import ClientError

# Initialize the Bedrock client
bedrock = boto3.client('bedrock', region_name='us-east-1')

# Job details
job_name = 'fine-tuning-titan-image-generation-job11'
model_name = 'fine_tuned_titan_image_model'
role_arn = 'arn:aws:iam::236515146491:role/TitanImageGenerationFineTunning'  # Replace with your actual role ARN

# Base model ID for the Titan Image Generation model
base_model_id = "amazon.titan-image-generator-v1:0"  # Ensure this is the correct model ID

# Training Data S3 URI
training_data_s3_uri = "s3://finetunningforimagegeneration/fine_tuning_images/updated_metadata.jsonl"

# Output Data S3 URI
output_data_s3_uri = "s3://finetunningforimagegeneration/output/"

# Number of steps based on 23 images
num_steps = 4000  # Setting the number of steps based on AWS recommendations for 23 images

# Hyperparameters based on AWS recommendations
hyperparameters = {
    "stepCount": str(num_steps),  # Number of steps
    "batchSize": "8",         # Batch size
    "learningRate": "0.00001" # Learning rate
}

# Create model customization job
try:
    response = bedrock.create_model_customization_job(
        customizationType="FINE_TUNING",
        jobName=job_name,
        customModelName=model_name,
        roleArn=role_arn,
        baseModelIdentifier=base_model_id,
        hyperParameters=hyperparameters,
        trainingDataConfig={"s3Uri": training_data_s3_uri},
        outputDataConfig={"s3Uri": output_data_s3_uri}  # Output S3 path
    )
    print("Model customization job created:", response)
except ClientError as error:
    print("Error creating model customization job:", error)


Model customization job created: {'ResponseMetadata': {'RequestId': '35c9b428-e4d9-42fc-889c-4c077c3069ca', 'HTTPStatusCode': 201, 'HTTPHeaders': {'date': 'Tue, 30 Jul 2024 16:21:42 GMT', 'content-type': 'application/json', 'content-length': '122', 'connection': 'keep-alive', 'x-amzn-requestid': '35c9b428-e4d9-42fc-889c-4c077c3069ca'}, 'RetryAttempts': 1}, 'jobArn': 'arn:aws:bedrock:us-east-1:236515146491:model-customization-job/amazon.titan-image-generator-v1:0/ehi8czjjmgzm'}


In [1]:
import time
import boto3
bedrock = boto3.client('bedrock')

In [2]:
status = bedrock.get_model_customization_job(jobIdentifier="fine-tuning-titan-image-generation-job11")["status"]

print("status checking ", status)

status checking  Failed
